## **CNN Modeling**

In [24]:
import numpy as np
import pandas as pd

# 벡터화된 데이터 불러오기
data = pd.read_json("./labeled_data.json")
data.columns = ["label", "trigram"]

In [25]:
# flastten: 3 x 50 -> 150
data['trigram'] = data['trigram'].apply(lambda x: (np.array(x).reshape(-1)))

In [26]:
# train/test 데이터 분리
from sklearn.model_selection import train_test_split

y = data.pop('label')
x = data

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [27]:
x_train = np.array(x_train['trigram'].tolist())
x_test = np.array(x_test['trigram'].tolist())
y_train = y_train.values
y_test = y_test.values

In [28]:
# row x embeedding 
x_train.shape

(9779, 150)

In [29]:
# row x 1
y_train.shape

(9779,)

In [30]:
# keras input
def reshape(df, dim):
    return df.reshape(df.shape[0], dim, 1)

In [31]:
x_train = reshape(x_train, 150)
x_test = reshape(x_test, 150)
x_train.shape

(9779, 150, 1)

### [1DCNN]

In [35]:
# f1 score를 알기 위한 함수
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))